[Reference](https://anmol3015.medium.com/say-goodbye-to-matplotlib-and-seaborn-for-your-python-plots-5cade2f7fba2)

In [1]:
# import plotly packages 
import plotly.graph_objects as go
import plotly.express as px

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/shinokada/covid-19-stats/master/data/daily-new-confirmed-cases-of-covid-19-tests-per-case.csv')
display(df.head())
display(df.shape)

,Entity,Code,Date,Daily confirmed cases (7-day average),Days since confirmed cases first reached 30 per day
0,Afghanistan,AFG,"Jan 2, 2020",0.0,-98.0
1,Afghanistan,AFG,"Jan 3, 2020",0.0,-97.0
2,Afghanistan,AFG,"Jan 4, 2020",0.0,-96.0
3,Afghanistan,AFG,"Jan 5, 2020",0.0,-95.0
4,Afghanistan,AFG,"Jan 6, 2020",0.0,-94.0


(26123, 5)

In [3]:
## setting shorter names of the columns 
df.columns = ['Countries','Code','Date','Avg Daily Confirmed',
                'Days to first 30 Cases']

## excluding data aggregated at world, continent, and other groupings.
excludes = ["World", "Africa", "North America", "South America", "Asia", "Europe", "European Union", "High income", "Low income", "Lower middle income", "Oceania", "Upper middle income", "World excl. China", "World excl. China and South Korea", "International",            "World excl. China, South Korea, Japan and Singapore","Asia excl. China"] 
df=df[~df['Countries'].isin(excludes)]
df

## converting column from string to date
df['Date']= pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

<ipython-input-3-7da42c1babbc>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date']= pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')


In [4]:
## to get the date on which 30 cases were reached for the 1st time
date_df = df[df['Days to first 30 Cases']==0]

## creating scatter plot with plotly
fig = px.scatter(date_df, x = 'Date', y = 'Avg Daily Confirmed',
                     hover_name = 'Countries',color = 'Countries')

fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
'title': 'Date of crossing first 30 Cases'})

fig.show()

In [5]:
fig = px.line(df, x="Date", y="Avg Daily Confirmed", color="Countries")

fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
'title': 'Count of Avg. Confirmed Cases'})
fig.show()

In [6]:
total_confirmed = df.groupby(['Countries'])['Avg Daily Confirmed'].sum().reset_index()
total_confirmed.columns = ['Countries','Total Confirmed Cases']
top_confirmed = total_confirmed.sort_values(by = ['Total Confirmed Cases'],ascending = False)[:10]


fig = px.bar(top_confirmed, x = 'Countries', y = 'Total Confirmed Cases')

fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
'title': 'Countries with most number of Confirmed Cases'})

fig.show()

In [7]:
### 1. Preparing the data for animation - countries with most cases
includes = ['United States','Russia', 'India','Brazil','United Kingdom']
includes = ['United States']
df_mar_may = df[(df['Date']>'2020-03-01') & (df['Date'] <'2020-06-14')]
df_mar_may.columns = ['Countries','Code','Date','confirmed','days_since_confirmed']
df_us=df_mar_may[df_mar_may['Countries'].isin(includes)]

includes = ['India']
df_ind=df_mar_may[df_mar_may['Countries'].isin(includes)]

includes = ['Brazil']
df_brazil=df_mar_may[df_mar_may['Countries'].isin(includes)]

includes = ['Russia']
df_rus=df_mar_may[df_mar_may['Countries'].isin(includes)]

includes = ['United Kingdom']
df_uk=df_mar_may[df_mar_may['Countries'].isin(includes)]


### creating the animation 
import plotly.offline as pyo


## 2. setting the layout
fig = go.Figure(
       layout=go.Layout(
        xaxis=dict(range=[-5, 100], autorange=False),
        yaxis=dict(range=[7, 31950], autorange=False),
        title="Confirmed Covid Cases (2nd March'20 - 13th June'20)",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                         args = [None] )])]
)
) #2

fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
'xaxis_title': 'Days since confirmed cases first reached 30 per day',
'yaxis_title': "Daily confirmed cases (7-day average)" })

# 3. Add traces - one line for each country
init = 1

fig.add_trace(
    go.Scatter(x=df_us.days_since_confirmed[:init],
               y=df_us.confirmed[:init],
               name="US",
               visible=True,
               line=dict(color="#33CFA5", )))


fig.add_trace(
    go.Scatter(x=df_ind.days_since_confirmed[:init],
               y=df_ind.confirmed[:init],
               name="India",
               visible=True,
               line=dict(color="#bf00ff"),))

fig.add_trace(
    go.Scatter(x=df_brazil.days_since_confirmed[:init],
               y=df_brazil.confirmed[:init],
               name="Brazil",
               visible=True,
               line=dict(color="#33C7FF"),
              ))

fig.add_trace(
    go.Scatter(x=df_rus.days_since_confirmed[:init],
               y=df_rus.confirmed[:init],
               name="Russia",
               visible=True,
               line=dict(color="#FF5733"),
              ))

fig.add_trace(
    go.Scatter(x=df_uk.days_since_confirmed[:init],
               y=df_uk.confirmed[:init],
               name="United Kingdom",
               visible=True,
               line=dict(color="#6E6E6E"),
              ))

# 4. Animation
fig.update(frames=[
    go.Frame(
        data=[
            go.Scatter(x=df_us.days_since_confirmed[:k], y=df_us.confirmed[:k]),
            go.Scatter(x=df_ind.days_since_confirmed[:k], y=df_ind.confirmed[:k]),
         go.Scatter(x=df_brazil.days_since_confirmed[:k], y=df_brazil.confirmed[:k]),
            go.Scatter(x=df_rus.days_since_confirmed[:k], y=df_rus.confirmed[:k]),
            go.Scatter(x=df_uk.days_since_confirmed[:k], y=df_uk.confirmed[:k]),
        ]
        
    )
    for k in range(init, len(df_us)+1)])

# Extra Formatting
fig.update_xaxes(ticks="outside", tickwidth=2, tickcolor='white', ticklen=10)
fig.update_yaxes(ticks="outside", tickwidth=2, tickcolor='white', ticklen=1)
fig.update_layout(yaxis_tickformat=',')
fig.update_layout(legend=dict(x=0, y=1.1), legend_orientation="h")


fig.show()